In [1]:
from dataclasses import dataclass
from typing import Union, Type, Any

In [2]:
STANDARD_UNIT_PREFIXES = {
    'n': 1e-9,
    'u': 1e-6,
    'm': 1e-3,
    'c': 1e-2,
    'd': 1e-1,
    '': 1,
    'k': 1e3,
    'M': 1e6,
    'G': 1e9,
    'T': 1e12,
}

In [3]:
@dataclass 
class Unit:
    """
    Класс единиц измерений
    """
    m: int
    s: int
    g: int
    Hz: int
    B: int

    def __mul__(self, other):
        """
        Операция перемножения степеней 
        """
        u = Unit(m= self.m + other.m,
                 s= self.s + other.s,
                 g= self.g + other.g,
                 Hz= self.Hz + other.Hz,
                 B= self.B + other.B)
        
        return u
    
    def __eq__(self, other):
        
        m = self.m == other.m
        s = self.s == other.s
        g = self.g == other.g
        Hz = self.Hz == other.Hz
        B = self.B == other.B
        
        return m and s and g and Hz and B
    
    def inverse(self):
        """
        Операция обращение степеней
        Пример: kg^3 s^2 -> kg^-3 s^-2
        """
        u = Unit(m= -self.m,
                 s= -self.s,
                 g= -self.g,
                 Hz= -self.Hz,
                 B= -self.B)
        
        return u

In [4]:
@dataclass 
class Variable:
    """
    Класс переменной 
    """
    name: ''
    value: Union[int, float, bool, str]
    unit: Unit
    description: ''
        
    def __add__(self, other):
        """
        Сложение класса Variable
        
        Знак операции +
        """
        
        assert(self.unit == other.unit)
        
        var = Variable(name='',
                       value=self.value + other.value,
                       unit=self.unit,
                       description=f'Sum of {self.name} and {other.name}' )
        
        return var
    
    def __sub__(self, other):
        """
        Вычитание класса Variable
        
        Знак операции -
        """

        
        assert(self.unit == other.unit)

        
        var = Variable(name='',
                       value=self.value - other.value,
                       unit=self.unit,
                       description=f'Difference of {self.name} and {other.name}' )
        return var
    
    def __mul__(self, other):
        """
        Умножение класса Variable
        
        Знак операции *
        """
        
        u = self.unit * other.unit
        var = Variable(name='',
                       value=self.value * other.value,
                       unit=u,
                       description=f'The product of {self.name} and {other.name}')
        
        return var
    
    def __truediv__(self, other):
        """
        Деление класса Variable
        
        Знак операции /
        """
                
        u = self.unit * Unit.inverse(other.unit)
        
        var = Variable(name='',
                       value= self.value/other.value,
                       unit=u,
                       description=f'Fraction of {self.name} and {other.name}')
        
        return var
        
    def __neg__(self):
        """
        Операция -
        
        """
        
        var = Variable(name='',
                       value= -self.value,
                       unit=self.unit,
                       description=f'Neg of {self.name}')
        return var
    
    def __str__(self):
        s = f'Name: {self.name}\n'
        s += f'Value: {self.value}\n'
        s += f'Power of unit: m={self.unit.m}, s={self.unit.s}, g={self.unit.g}, Hz={self.unit.Hz}, B={self.unit.B}\n'
        s += f'Description: {self.description}\n'
        return s

In [5]:
@dataclass 
class Setting:
    """
    Класс настройки
    """
    name: ''
    default_value: Variable
    description: ''

In [6]:
dict_units = {
        'nm': -9, 'um': -6, 'mm': -3, 'cm': -2, 'dm': -1 ,'m': 1, 'km': 3,'Mm': 6,'Gm': 9,'Tm': 12,
        'ns': -9, 'us': -6, 'ms': -3, 'cs': -2, 'ds': -1 ,'s': 1, 'ks': 3,'Ms': 6,'Gs': 9,'Ts': 12,
        'ng': -9, 'ug': -6, 'mg': -3, 'cg': -2, 'dg': -1 ,'g': 1, 'kg': 3,'Mg': 6,'Gg': 9,'Tg': 12,
        'nHz':-9, 'uHz':-6, 'mHz':-3, 'cHz':-2, 'dHz':-1 ,'Hz':1, 'kHz':3,'MHz':6,'GHz':9,'THz':12,
        'nB': -9, 'uB': -6, 'mB': -3, 'cB': -2, 'dB': -1 ,'B': 1, 'kB': 3,'MB': 6,'GB': 9,'TB': 12}

In [7]:
def input_setting(s):
    
    d = {'m':0, 's':0, 'g':0, 'Hz':0, 'B':0}
        
    if '[' in s:
        s_idx = s.find('[')
        f_idx = s.find(']')
        splited_string = s[s_idx+1:f_idx].split()
        value = float(s[:s_idx])
    else:
        value = float(s)
        u = Unit(d['m'], d['s'], d['g'], d['Hz'], d['B'] )
    
        var = Variable(name='',
              value=value, 
              unit=u,
              description='')
        
        return var
    
    
    for i in splited_string:
        if (i.find('^') == -1):
            if (i.find('s') != -1):
                d['s'] += dict_units[i]
            elif (i.find('g') != -1):
                d['g'] += dict_units[i] 
            elif (i.find('Hz') != -1):
                d['Hz'] += dict_units[i] 
            elif (i.find('B') != -1):
                d['B'] += dict_units[i] 
            else:
                d['m'] += dict_units[i]
                
        else:
            i = i.split('^')
            if (i[0].find('s') != -1):
                d['s'] += dict_units[i[0]] * int(i[-1])
            elif (i[0].find('g') != -1):
                d['g'] += dict_units[i[0]] * int(i[-1])
            elif (i[0].find('Hz') != -1):
                d['Hz'] += dict_units[i[0]] * int(i[-1])
            elif (i[0].find('B') != -1):
                d['B'] += dict_units[i[0]] * int(i[-1])
            else:
                d['m'] += dict_units[i[0]] * int(i[-1])
                
    
    u = Unit(d['m'], d['s'], d['g'], d['Hz'], d['B'] )
    
    var = Variable(name='',
          value=value, 
          unit=u,
          description='')
    
    
    return var

In [8]:
OPERATORS = {'+': (1, lambda x, y: x + y), '-': (1, lambda x, y: x - y),
             '*': (2, lambda x, y: x * y), '/': (2, lambda x, y: x / y)}


def string_to_var(formula):
    
    def parse(formula_string):
        number = ''
        for s in formula_string:
            if s in '1234567890.':
                number += s
            elif s in '[]^nmucdkMGTsgHzB':
                number += s
            elif number:
                yield number
                number = ''
            if s in OPERATORS or s in "()":
                yield s
        if number:
            yield number
            
    def gen_to_list(parsed_formula):
        st = []
        s = ''
        parsed_formula = list(parsed_formula)
        for i in range(len(parsed_formula)-1):
            if parsed_formula[i] in '()+-/*':
                if s != '':
                    st.append(s)
                st.append(parsed_formula[i])
                s = ''
            else:
                s += parsed_formula[i] + ' '
        s += parsed_formula[-1]
        st.append(s)
        return st 
    
    def shunting_yard(parsed_formula):
        stack = []
        arr = gen_to_list(parsed_formula) 
        for token in arr:
            if token in OPERATORS:
                while stack and stack[-1] != "(" and OPERATORS[token][0] <= OPERATORS[stack[-1]][0]:
                    yield stack.pop()
                stack.append(token)
            elif token == ")":
                while stack:
                    x = stack.pop()
                    if x == "(":
                        break
                    yield x
            elif token == "(":
                stack.append(token)
            else:
                yield input_setting(token)
        while stack:
            yield stack.pop()

    def calc(polish):
        stack = []
        for token in polish:
            if type(token) ==  str:
                y, x = stack.pop(), stack.pop()
                stack.append(OPERATORS[token][1](x, y))
            else:
                stack.append(token)
        return stack[0]

    return calc(shunting_yard(parse(formula)))